Defino los mapas de los conjuntos y los alias para los 4 algoritmos:

In [ ]:
import pandas as pd
import os
import joblib  # Para guardar los modelos entrenados
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# --- CONFIGURACIÓN ---

# Definimos los métodos (algoritmos) que vamos a usar
# Es importante activar probability=True en SVM para el futuro cálculo de Ensembles
modelos_config = {
    "knn": KNeighborsClassifier(n_neighbors=3), # Puedes ajustar n_neighbors
    "svm": SVC(probability=True, random_state=42),
    "nb": GaussianNB(),
    "rf": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Mapeo de los nombres de dataset a tus carpetas y sufijos de archivo
# Según lo que generamos en la práctica anterior:
datasets_map = {
    # Datos Originales
    "original": {"folder": "Original", "suffix": ""},
    "originalPCA95": {"folder": "Original", "suffix": "_pca95"},
    "originalPCA80": {"folder": "Original", "suffix": "_pca80"},
    
    # Datos Estandarizados
    "estandarizado": {"folder": "Estandarizado", "suffix": ""},
    "estandarizadoPCA95": {"folder": "Estandarizado", "suffix": "_pca95"},
    "estandarizadoPCA80": {"folder": "Estandarizado", "suffix": "_pca80"},
    
    # Datos Normalizados
    "normalizado": {"folder": "Normalizado", "suffix": ""},
    "normalizadoPCA95": {"folder": "Normalizado", "suffix": "_pca95"},
    "normalizadoPCA80": {"folder": "Normalizado", "suffix": "_pca80"},
}

# Carpeta donde guardaremos los modelos entrenados
models_output_folder = "models"
if not os.path.exists(models_output_folder):
    os.makedirs(models_output_folder)

Siguiente función:


-Construye la ruta del archivo training correcto basándose en el mapa que acabamos de definir arriba

-Carga los datos

-Separa lo de las características (X) y las etiquetas (y)

-Entrena el modelo que le entra

-Guarda el modelo en la carpeta models con su nombre ej estructura  --->  model_3_normalizadoPCA95_knn.pkl

In [ ]:
def train_and_save_model(fold, method, dataset_name):
    """
    Entrena un modelo específico para una iteración y dataset dados, y lo guarda en disco.
    
    Args:
        fold (int): Número de iteración (1 a 5).
        method (str): Nombre del algoritmo ('knn', 'svm', 'nb', 'rf').
        dataset_name (str): Clave del dataset (ej: 'original', 'normalizadoPCA80').
    """
    
    # 1. Identificar archivo de datos
    info = datasets_map[dataset_name]
    filename = f"training{fold}{info['suffix']}.csv"
    filepath = os.path.join(info['folder'], filename)
    
    # Verificar si existe
    if not os.path.exists(filepath):
        print(f"Error: No se encuentra {filepath}")
        return

    # 2. Cargar datos
    # Asumimos que la última columna es la etiqueta (clase)
    data = pd.read_csv(filepath, header=None)
    X_train = data.iloc[:, :-1].values
    y_train = data.iloc[:, -1].values.ravel() # ravel() para asegurar que sea un vector 1D

    # 3. Inicializar el modelo
    if method not in modelos_config:
        print(f"Método {method} no reconocido.")
        return
    
    # Usamos 'clone' o creamos una nueva instancia para asegurar que está limpio
    # Aquí simplemente tomamos la configuración base y la ajustamos si fuera necesario
    # Para simplificar, usamos la instancia del diccionario (se re-entrena con fit)
    clf = modelos_config[method]
    
    # 4. Entrenar
    clf.fit(X_train, y_train)
    
    # 5. Guardar el modelo
    # Nombre formato: model_{fold}_{dataset}_{method}.pkl
    model_filename = f"model_{fold}_{dataset_name}_{method}.pkl"
    model_path = os.path.join(models_output_folder, model_filename)
    
    joblib.dump(clf, model_path)
    # print(f" Modelo guardado: {model_filename}")